In [17]:
import numpy as np
import scipy.stats as sps
import pandas as pd
import plotly.graph_objs as go
import tqdm

np.random.seed(1488)

import warnings
warnings.filterwarnings("ignore")

# Задание 1: Длительность теста

**Реализовать формулу подсчета длительности теста**

In [12]:
def get_ttest_mannwhitneyutest_powers(dist, params_1, params_2, equal_var=True, alpha=0.05, n_exps=1000):
    p_values_less_counter_ttest = 0
    p_values_less_counter_mannwhitneyu = 0
    for i in tqdm.tqdm(range(n_exps)):
        x_a = dist(**params_1)
        x_b = dist(**params_2)
        p_value_ttest = sps.ttest_ind(x_a, x_b, equal_var=equal_var).pvalue
        p_value_mannwhitneyu = sps.mannwhitneyu(x_a, x_b).pvalue
        p_values_less_counter_ttest += (p_value_ttest < alpha)
        p_values_less_counter_mannwhitneyu += (p_value_mannwhitneyu < alpha)

    return p_values_less_counter_ttest / n_exps * 100, p_values_less_counter_mannwhitneyu / n_exps * 100

In [13]:
def get_qq_plot(p_values, title):
    p_values = np.array(p_values)
    probs = []
    x = [0.01 * i for i in range(101)]
    for i in range(101):
        alpha_step = 0.01 * i
        probs.append(p_values[p_values < alpha_step].shape[0] / p_values.shape[0])
    fig = go.Figure([go.Scatter(x=x, y=probs, mode="markers", name="p_value"),
                 go.Scatter(x=x, y=x, mode="lines", name="uniform")])
    fig.update_layout(height=600, width=600, title=title)
    return fig

In [14]:
def duration(k, delta_effect, sigma_1, sigma_2, alpha=0.05, beta=0.2):
    z = sps.norm.ppf(1 - alpha/2) + sps.norm.ppf(1-beta)
    n = (k+1) * z ** 2 * (sigma_1 ** 2 + sigma_2 **2 / k) / (delta_effect ** 2)
    return n

**Сравнить ее с онлайн калькуляторами. При сравнении оценить мощность критерия при указанном изменении и рассчитанном количестве наблюдений в выборке.**

In [18]:
powers = [0.8, 0.6]
alpha = 0.05
p = 0.6
effects = [0.01, 0.3]
for power, effect in zip(powers, effects):
    sigma_1 = np.sqrt(p * (1 - p))
    sigma_2 = np.sqrt((p + effect)  * (1 - p - effect))
    test_duration = duration(k=1, delta_effect=effect, sigma_1=sigma_1, sigma_2=sigma_2, alpha=alpha, beta=1 - power)
    ttest_power, mannwhitneyu_power = get_ttest_mannwhitneyutest_powers(sps.bernoulli.rvs,
                                                                        {"p": p, "size": int(test_duration/2)},
                                                                        {"p": p + effect, "size": int(test_duration/2)},
                                                                        alpha=alpha)
    print(f'\neffect: {effect}, power: {power}, duration: {test_duration}')
    print(f"ttest power: {ttest_power:.3f}, mannwhitneyu power: {mannwhitneyu_power:.3f}")
    print(f'expected_power: {power}\n')

100%|██████████| 1000/1000 [01:08<00:00, 14.62it/s]



effect: 0.01, power: 0.8, duration: 75019.59250090858
ttest power: 80.900, mannwhitneyu power: 80.900
expected_power: 0.8



100%|██████████| 1000/1000 [00:02<00:00, 433.70it/s]


effect: 0.3, power: 0.6, duration: 35.92413711941172
ttest power: 52.400, mannwhitneyu power: 52.400
expected_power: 0.6



![effect-1 power80](https://i.ibb.co/vwbKxvZ/Screenshot-from-2023-06-28-19-56-15.png)


![effect-30 power60](https://i.ibb.co/Tg2Lcqb/Screenshot-from-2023-06-28-19-56-53.png)

# Задание 2: Линеаризация

**Реализовать метод линеаризации.**

In [21]:
def linearization(p, effect):
    n_exps = 1000
    p_values = []
    p_values_lin = []
    for _ in range(n_exps):
        records = []
        for i in range(100):
            n_views = int(sps.expon.rvs(loc=100, scale=100))
            clicks = sps.bernoulli.rvs(p=p, size=n_views)
            records.append([n_views, np.sum(clicks), np.sum(clicks)/ n_views, "A"])
        for i in range(100):
            n_views = int(sps.expon.rvs(loc=100, scale=100))
            clicks = sps.bernoulli.rvs(p=p + effect, size=n_views)
            records.append([n_views, np.sum(clicks), np.sum(clicks)/ n_views, "B"])
        df_data = pd.DataFrame(records, columns=["views", "clicks", "cr", "group"])

        cr_A = df_data[df_data["group"] == "A"]["clicks"].sum() / df_data[df_data["group"] == "A"]["views"].sum()
        df_data["cr_lin"] = df_data["clicks"] - cr_A * df_data["views"]

        x_a = df_data[df_data["group"] == "A"]["cr"]
        x_b = df_data[df_data["group"] == "B"]["cr"]
        p_value = sps.ttest_ind(x_a, x_b).pvalue
        p_values.append(p_value)

        x_a_lin = df_data[df_data["group"] == "A"]["cr_lin"]
        x_b_lin = df_data[df_data["group"] == "B"]["cr_lin"]
        p_value_lin = sps.ttest_ind(x_a_lin, x_b_lin).pvalue
        p_values_lin.append(p_value_lin)
    return p_values, p_values_lin

**Проверить для него корректность и мощность. Мощность должна быть больше, чем просто на обычных значениях конверсии пользователей.**

In [ ]:
p_values, p_values_lin = linearization(0.05, 0)

In [25]:
get_qq_plot(p_values, title='vanilla')

In [26]:
get_qq_plot(p_values_lin, title='with linearization')

In [27]:
p_values, p_values_lin = linearization(0.05, 0.005)

p_values = np.array(p_values)
print(f'Power vanilla: {p_values[p_values < alpha].shape[0] / p_values.shape[0] * 100:.4f}')
p_values_lin = np.array(p_values_lin)
print(f'Power with linearization: {p_values_lin[p_values_lin < alpha].shape[0] / p_values_lin.shape[0] * 100:.4f}')

Power vanilla: 54.5000
Power with linearization: 60.1000


# Задание 3: CUPED

**Реализовать метод CUPED.**

In [29]:
def cuped(dist, pre_exp, params_1, params_2, size=10000):
    n_exps = 1000
    p_values = []
    p_values_cuped = []

    for _ in range(n_exps):
        df_A = pd.DataFrame()
        df_A["user"] = [f"A_{x:5}" for x in range(size)]
        df_A["pre_exp"] = pre_exp
        df_B = pd.DataFrame()
        df_B["pre_exp"] = pre_exp
        df_B["user"] = [f"B_{x:5}" for x in range(size)]
        df_A["payments"] = dist(**params_1) * df_A["pre_exp"]
        df_B["payments"] = dist(**params_2) * df_B["pre_exp"]

        p_values.append(sps.ttest_ind(df_A["payments"], df_B["payments"]).pvalue)

        x_a = df_A["pre_exp"]
        x_b = df_B["pre_exp"]
        y_a = df_A["payments"]
        y_b = df_B["payments"]
        theta = np.cov(x_a, y_a)[0,1] / np.std(x_a)**2

        df_A["payments_cuped"] = df_A["payments"] - theta * df_A["pre_exp"]
        df_B["payments_cuped"] = df_B["payments"] - theta * df_B["pre_exp"]

        p_values_cuped.append(sps.ttest_ind(df_A["payments_cuped"], df_B["payments_cuped"]).pvalue)
    return p_values, p_values_cuped

**Проверить для него корректность и мощность.**

In [30]:
size = 10000
pre_exp = sps.norm.rvs(loc=100, scale=20, size=size)
p_values, p_values_cuped = cuped(sps.expon.rvs, pre_exp,
                                 {"loc": 100, "scale": 100, "size": size},
                                 {"loc": 100, "scale": 100, "size": size},
                                 size=size)
get_qq_plot(p_values_cuped, title='Cuped')

In [ ]:
effect = 0.005
p_values, p_values_cuped = cuped(sps.expon.rvs, pre_exp,
                                 dict(loc=1, scale=0.1, size=size), dict(loc=1 + effect, scale=0.1, size=size))
p_values, p_values_cuped = cuped(sps.expon.rvs, pre_exp,
                                 {"loc": 1, "scale": 0.1, "size": size},
                                 {"loc": 1 + effect, "scale": 0.1, "size": size},
                                 size=size)

In [33]:
p_values = np.array(p_values)
print(f'Power vanilla test: {p_values[p_values < alpha].shape[0] / p_values.shape[0] * 100:.4f}')

Power vanilla test: 11.6000


In [34]:
p_values_cuped = np.array(p_values_cuped)
print(f'Power with CUPED: {p_values_cuped[p_values_cuped < alpha].shape[0] / p_values_cuped.shape[0] * 100:.4f}')

Power with CUPED: 92.8000
